In [ ]:
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import datetime
import joblib
import os

# Download required NLTK data
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

class AmbiguityDetector:
    def __init__(self):
        self.model = None
        self.vectorizer = TfidfVectorizer(max_features=5000)
        self.feature_names = None
        
    def extract_features(self, texts):
        """Extract features related to ambiguity from text."""
        features = []
        
        for text in texts:

            doc = nlp(text)
            

            text_features = {}
            
            # 1. Lexical ambiguity 
            polysemy_count = 0
            content_words = 0
            
            for token in doc:
                if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'VERB', 'ADJ']:
                    content_words += 1
                    synsets = wordnet.synsets(token.text)
                    if len(synsets) > 1:
                        polysemy_count += 1
            
            text_features['polysemy_ratio'] = polysemy_count / max(1, content_words)
            
            # 2. Referential ambiguity 
            pronouns = [token for token in doc if token.pos_ == 'PRON']
            nouns = [token for token in doc if token.pos_ in ['NOUN', 'PROPN']]
            text_features['pronoun_count'] = len(pronouns)
            text_features['pronoun_noun_ratio'] = len(pronouns) / max(1, len(nouns))
            
            # 3. Vague quantifiers
            vague_quantifiers = ['some', 'many', 'few', 'several', 'various', 'numerous', 'lot', 'lots']
            vague_count = sum(1 for token in doc if token.text.lower() in vague_quantifiers)
            text_features['vague_quantifier_count'] = vague_count
            
            # 4. Question words without specific context
            question_words = ['who', 'what', 'where', 'when', 'why', 'how']
            question_word_count = sum(1 for token in doc if token.text.lower() in question_words)
            text_features['question_word_count'] = question_word_count
            
            # 5. Sentence complexity measures
            text_features['avg_token_length'] = np.mean([len(token.text) for token in doc if token.is_alpha])
            text_features['sentence_count'] = len(list(doc.sents))
            text_features['avg_sentence_length'] = len(doc) / max(1, text_features['sentence_count'])
            
            # 6. Context specificity (measure of named entities)
            named_entities = len(doc.ents)
            text_features['named_entity_count'] = named_entities
            
            # 7. Conjunction count (potential for ambiguous scope)
            conjunctions = sum(1 for token in doc if token.pos_ == 'CCONJ')
            text_features['conjunction_count'] = conjunctions
            
            features.append(text_features)
            
        return pd.DataFrame(features)
    
    def train(self, texts, labels):
        """Train the ambiguity detection model."""

        X_tfidf = self.vectorizer.fit_transform(texts)
        

        X_custom = self.extract_features(texts)
        

        X_tfidf_array = X_tfidf.toarray()
        X_combined = np.hstack([X_tfidf_array, X_custom.values])
        

        X_train, X_val, y_train, y_val = train_test_split(
            X_combined, labels, test_size=0.2, random_state=42
        )
        

        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)
        

        y_pred = self.model.predict(X_val)
        print(classification_report(y_val, y_pred))
        
        return self
    
    def predict(self, text):
        """Predict if a text is ambiguous."""
        if isinstance(text, str):
            text = [text]
            

        X_tfidf = self.vectorizer.transform(text)
        X_custom = self.extract_features(text)
        

        X_tfidf_array = X_tfidf.toarray()
        X_combined = np.hstack([X_tfidf_array, X_custom.values])
        

        return self.model.predict(X_combined), self.model.predict_proba(X_combined)
    
    def analyze_ambiguity(self, text):
        """Analyze why a text might be ambiguous and return explanation."""
        doc = nlp(text)
        
        ambiguity_types = []
        explanations = []
        
        # Check for lexical ambiguity
        polysemous_words = []
        for token in doc:
            if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'VERB', 'ADJ']:
                synsets = wordnet.synsets(token.text)
                if len(synsets) > 2:  # Only consider highly ambiguous words
                    polysemous_words.append(token.text)
        
        if polysemous_words:
            ambiguity_types.append("Lexical Ambiguity")
            explanations.append(f"Words with multiple meanings: {', '.join(polysemous_words[:3])}")
        
        # Check for referential ambiguity
        pronouns = [token.text for token in doc if token.pos_ == 'PRON']
        if len(pronouns) > 2 and len([t for t in doc if t.pos_ in ['NOUN', 'PROPN']]) > 2:
            ambiguity_types.append("Referential Ambiguity")
            explanations.append(f"Multiple pronouns with potential unclear references: {', '.join(pronouns[:3])}")
        
        # Check for vague quantifiers
        vague_terms = ['some', 'many', 'few', 'several', 'various', 'numerous', 'lot', 'lots']
        vague_found = [token.text for token in doc if token.text.lower() in vague_terms]
        if vague_found:
            ambiguity_types.append("Vague Quantifiers")
            explanations.append(f"Vague quantity terms: {', '.join(vague_found)}")
        
        # Check for missing information
        question_words = ['who', 'what', 'where', 'when', 'why', 'how']
        question_word_found = [token.text for token in doc if token.text.lower() in question_words]
        if question_word_found and len(doc) < 10:
            ambiguity_types.append("Incomplete Information")
            explanations.append("Query appears too short and may lack sufficient context")
        
        return {
            "ambiguity_types": ambiguity_types,
            "explanations": explanations,
            "overall_assessment": "Ambiguous" if ambiguity_types else "Clear"
        }
    
    def save_model(self, filepath=None):
        """
        Save the trained model to disk.
        
        Args:
            filepath (str, optional): Path to save the model. If None, generates a timestamped filename.
        
        Returns:
            str: Path where the model was saved
        """
        if not self.model:
            raise ValueError("No trained model to save. Train the model first.")

        if filepath is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filepath = f"ambiguity_detector_{timestamp}.pkl"
        

        directory = os.path.dirname(filepath)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)
        

        model_data = {
            'model': self.model,
            'vectorizer': self.vectorizer,
            'feature_names': self.feature_names
        }

        joblib.dump(model_data, filepath)
        print(f"Model saved to {filepath}")
        
        return filepath
    
    @classmethod
    def load_model(cls, filepath):
        """
        Load a trained model from disk.
        
        Args:
            filepath (str): Path to the saved model file
            
        Returns:
            AmbiguityDetector: Loaded model instance
        """
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Model file not found: {filepath}")
        

        try:
            model_data = joblib.load(filepath)

            instance = cls()

            instance.model = model_data['model']
            instance.vectorizer = model_data['vectorizer']
            instance.feature_names = model_data['feature_names']
            
            print(f"Model loaded from {filepath}")
            return instance
            
        except Exception as e:
            raise ValueError(f"Error loading model: {e}")



def generate_training_data(n_samples=1000):

    clear_queries = [
        "What is the capital of France?",
        "How do I convert a pandas DataFrame to a CSV file?",
        "Show me the recipe for chocolate chip cookies.",
        "Explain the process of photosynthesis in detail.",
        "What was the significance of the Magna Carta?",
        "Write a Python function to calculate Fibonacci numbers.",
        "Summarize the plot of The Great Gatsby by F. Scott Fitzgerald.",
        "List five exercises for strengthening core muscles.",
        "What temperature should I cook chicken to ensure it's safe?",
        "Explain the difference between HTTP and HTTPS protocols."
    ]
    
    ambiguous_queries = [
        "Why is it doing that?",
        "Can you help me with this?",
        "Is it good?",
        "What do they mean?",
        "How does it work?",
        "Can you tell me more about that topic?",
        "Why isn't it working?",
        "What's the best one?",
        "How do I fix it?",
        "When should I use it?"
    ]
    

    np.random.seed(42)
    clear_expanded = []
    for query in clear_queries:
        clear_expanded.append(query)
        words = query.split()
        for _ in range(9):  # Create 9 variations of each template

            if len(words) > 5:
                modified = ' '.join(words[:len(words)-np.random.randint(1, 3)]) + ' ' + ' '.join(words[-np.random.randint(1, 3):])
                clear_expanded.append(modified)
            else:
                clear_expanded.append(query)
    
    ambiguous_expanded = []
    for query in ambiguous_queries:
        ambiguous_expanded.append(query)
        for _ in range(9):  # Create 9 variations of each template
            # Might add filler words but keep ambiguous
            if np.random.random() > 0.5:
                fillers = ['um', 'like', 'so', 'you know', 'I mean']
                ambiguous_expanded.append(np.random.choice(fillers) + ' ' + query)
            else:
                ambiguous_expanded.append(query)
    

    all_texts = clear_expanded + ambiguous_expanded
    labels = [0] * len(clear_expanded) + [1] * len(ambiguous_expanded)
    

    indices = np.arange(len(all_texts))
    np.random.shuffle(indices)
    
    return [all_texts[i] for i in indices[:n_samples]], [labels[i] for i in indices[:n_samples]]


def main():

    texts, labels = generate_training_data(500)
    
    # Train ambiguity detector
    detector = AmbiguityDetector()
    detector.train(texts, labels)

    detector.save_model("model.pkl")
    

    test_queries = [
        "What is the capital of France?",
        "How do I cook pasta properly?",
        "Why isn't it working?",
        "Can you help me with this thing?",
        "What do they mean by that?"
    ]
    
    for query in test_queries:
        prediction, probabilities = detector.predict(query)
        ambiguity_analysis = detector.analyze_ambiguity(query)
        
        print(f"\nQuery: '{query}'")
        print(f"Prediction: {'Ambiguous' if prediction[0] == 1 else 'Clear'}")
        print(f"Confidence: {max(probabilities[0]) * 100:.2f}%")
        
        if ambiguity_analysis['ambiguity_types']:
            print("Ambiguity Analysis:")
            for i, (ambiguity_type, explanation) in enumerate(zip(
                ambiguity_analysis['ambiguity_types'], 
                ambiguity_analysis['explanations']
            )):
                print(f"  {i+1}. {ambiguity_type}: {explanation}")


if __name__ == "__main__":
    main()

[nltk_data] Downloading package wordnet to /Users/devshah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/devshah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/devshah/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /Users/devshah/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00        18

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Model saved to model.pkl

Query: 'What is the capital of France?'
Prediction: Clear
Confidence: 100.00%
Ambiguity Analysis:
  1. Lexical Ambiguity: Words with multiple meanings: capital
  2. Incomplete Information: Query appears too short and may lack sufficient context

Query: 'How do I cook pasta properly?'
Prediction: Ambiguous
Confidence: 82.00%
Ambiguity Analysis:
  1. Lexical Ambiguity: Words with multiple meanings: cook
  2. Incomplete Information: Query appears too short and may lack sufficient context

Query: 'Why isn't it working?'
Prediction: Ambiguous
Confidence: 100.00%
Ambiguity Analysis:
  1. Lexical Ambiguity: Words with multiple meanings: working
 

In [9]:
query = 'im confused'

detector = AmbiguityDetector.load_model("model.pkl")

prediction, probabilities = detector.predict(query)
ambiguity_analysis = detector.analyze_ambiguity(query)
        
print(f"\nQuery: '{query}'")
print(f"Prediction: {'Ambiguous' if prediction[0] == 1 else 'Clear'}")
print(f"Confidence: {max(probabilities[0]) * 100:.2f}%")
        
if ambiguity_analysis['ambiguity_types']:
    print("Ambiguity Analysis:")
    for i, (ambiguity_type, explanation) in enumerate(zip(
        ambiguity_analysis['ambiguity_types'], 
        ambiguity_analysis['explanations']
            )):
        print(f"  {i+1}. {ambiguity_type}: {explanation}")

Model loaded from model.pkl

Query: 'im confused'
Prediction: Ambiguous
Confidence: 91.00%
Ambiguity Analysis:
  1. Lexical Ambiguity: Words with multiple meanings: confused


In [ ]:
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import datetime
import joblib
import os

# Download required NLTK data
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

class AmbiguityDetector:
    def __init__(self):
        self.model = None
        self.vectorizer = TfidfVectorizer(max_features=5000)
        self.feature_names = None

    def extract_features(self, texts):
        """Extract features related to ambiguity from text."""
        features = []
        
        for text in texts:

            doc = nlp(text)
            

            text_features = {}
            
            # 1. Lexical ambiguity 
            polysemy_count = 0
            content_words = 0
            
            for token in doc:
                if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'VERB', 'ADJ']:
                    content_words += 1
                    synsets = wordnet.synsets(token.text)
                    if len(synsets) > 1:
                        polysemy_count += 1
            
            text_features['polysemy_ratio'] = polysemy_count / max(1, content_words)
            
            # 2. Referential ambiguity 
            pronouns = [token for token in doc if token.pos_ == 'PRON']
            nouns = [token for token in doc if token.pos_ in ['NOUN', 'PROPN']]
            text_features['pronoun_count'] = len(pronouns)
            text_features['pronoun_noun_ratio'] = len(pronouns) / max(1, len(nouns))
            
            # 3. Vague quantifiers
            vague_quantifiers = ['some', 'many', 'few', 'several', 'various', 'numerous', 'lot', 'lots']
            vague_count = sum(1 for token in doc if token.text.lower() in vague_quantifiers)
            text_features['vague_quantifier_count'] = vague_count
            
            # 4. Question words without specific context
            question_words = ['who', 'what', 'where', 'when', 'why', 'how']
            question_word_count = sum(1 for token in doc if token.text.lower() in question_words)
            text_features['question_word_count'] = question_word_count
            
            # 5. Sentence complexity measures
            text_features['avg_token_length'] = np.mean([len(token.text) for token in doc if token.is_alpha])
            text_features['sentence_count'] = len(list(doc.sents))
            text_features['avg_sentence_length'] = len(doc) / max(1, text_features['sentence_count'])
            
            # 6. Context specificity (measure of named entities)
            named_entities = len(doc.ents)
            text_features['named_entity_count'] = named_entities
            
            # 7. Conjunction count (potential for ambiguous scope)
            conjunctions = sum(1 for token in doc if token.pos_ == 'CCONJ')
            text_features['conjunction_count'] = conjunctions

            text_features['syntatic_ambiguity'] = self.detect_syntactic_ambiguity(text)
            text_features['modal_verb_count'] = self.detect_modal_verbs(text)
            
            features.append(text_features)
            
        return pd.DataFrame(features)


    def detect_syntactic_ambiguity(self, text):
        doc = nlp(text)
        for sent in doc.sents:
            for token in sent:
                if token.dep_ in ('prep', 'pobj') and token.head.pos_ == 'VERB':
                    return True  # Potential syntactic ambiguity
        return False
        
    def detect_modal_verbs(self, text):
        modal_verbs = {'can', 'could', 'may', 'might', 'shall', 'should', 'will', 'would', 'must'}
        doc = nlp(text)
        return sum(1 for token in doc if token.text.lower() in modal_verbs)


    
    def train(self, texts, labels):
        """Train the ambiguity detection model."""

        X_tfidf = self.vectorizer.fit_transform(texts)
        

        X_custom = self.extract_features(texts)
        

        X_tfidf_array = X_tfidf.toarray()
        X_combined = np.hstack([X_tfidf_array, X_custom.values])
        

        X_train, X_val, y_train, y_val = train_test_split(
            X_combined, labels, test_size=0.2, random_state=42
        )
        

        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)
        

        y_pred = self.model.predict(X_val)
        print(classification_report(y_val, y_pred))
        
        return self
    
    def predict(self, text):
        """Predict if a text is ambiguous."""
        if isinstance(text, str):
            text = [text]
            

        X_tfidf = self.vectorizer.transform(text)
        X_custom = self.extract_features(text)
        

        X_tfidf_array = X_tfidf.toarray()
        X_combined = np.hstack([X_tfidf_array, X_custom.values])
        

        return self.model.predict(X_combined), self.model.predict_proba(X_combined)
    
    def analyze_ambiguity(self, text):
        """Analyze why a text might be ambiguous and return explanation."""
        doc = nlp(text)
        
        ambiguity_types = []
        explanations = []
        
        # Check for lexical ambiguity
        polysemous_words = []
        for token in doc:
            if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'VERB', 'ADJ']:
                synsets = wordnet.synsets(token.text)
                
                #---------------- added synonym checking

                synonyms = set()
                for syn in synsets:
                    synonyms.update(syn.lemma_names())

                #-------------------------
                
                
                if len(synsets) > 2 and len(synonyms) > 4:  # Only consider highly ambiguous words
                    polysemous_words.append(token.text)
        
        if polysemous_words:
            ambiguity_types.append("Lexical Ambiguity")
            explanations.append(f"Words with multiple meanings: {', '.join(polysemous_words[:3])}")
        
        # Check for referential ambiguity
        pronouns = [token.text for token in doc if token.pos_ == 'PRON']
        if len(pronouns) > 2 and len([t for t in doc if t.pos_ in ['NOUN', 'PROPN']]) > 2:
            ambiguity_types.append("Referential Ambiguity")
            explanations.append(f"Multiple pronouns with potential unclear references: {', '.join(pronouns[:3])}")
        
        # Check for vague quantifiers
        vague_terms = ['some', 'many', 'few', 'several', 'various', 'numerous', 'lot', 'lots', 'tons', 'a bunch', 'plenty'
                       'a couple', 'a handful', 'loads', 'countless', 'a great deal', 'a good amount']
        vague_found = [token.text for token in doc if token.text.lower() in vague_terms]
        if vague_found:
            ambiguity_types.append("Vague Quantifiers")
            explanations.append(f"Vague quantity terms: {', '.join(vague_found)}")
        
        # Check for missing information
        question_words = ['who', 'what', 'where', 'when', 'why', 'how', 'which', "whose", "whom", "whenever", 
                          "wherever", "whichever", "whatever", "however"]
        question_word_found = [token.text for token in doc if token.text.lower() in question_words]
        if question_word_found and len(doc) < 10:
            ambiguity_types.append("Incomplete Information")
            explanations.append("Query appears too short and may lack sufficient context")
        
        return {
            "ambiguity_types": ambiguity_types,
            "explanations": explanations,
            "overall_assessment": "Ambiguous" if ambiguity_types else "Clear"
        }
    
    def save_model(self, filepath=None):
        """
        Save the trained model to disk.
        
        Args:
            filepath (str, optional): Path to save the model. If None, generates a timestamped filename.
        
        Returns:
            str: Path where the model was saved
        """
        if not self.model:
            raise ValueError("No trained model to save. Train the model first.")

        if filepath is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filepath = f"ambiguity_detector_{timestamp}.pkl"
        

        directory = os.path.dirname(filepath)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)
        

        model_data = {
            'model': self.model,
            'vectorizer': self.vectorizer,
            'feature_names': self.feature_names
        }

        joblib.dump(model_data, filepath)
        print(f"Model saved to {filepath}")
        
        return filepath
    
    @classmethod
    def load_model(cls, filepath):
        """
        Load a trained model from disk.
        
        Args:
            filepath (str): Path to the saved model file
            
        Returns:
            AmbiguityDetector: Loaded model instance
        """
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Model file not found: {filepath}")
        

        try:
            model_data = joblib.load(filepath)

            instance = cls()

            instance.model = model_data['model']
            instance.vectorizer = model_data['vectorizer']
            instance.feature_names = model_data['feature_names']
            
            print(f"Model loaded from {filepath}")
            return instance
            
        except Exception as e:
            raise ValueError(f"Error loading model: {e}")



def generate_training_data(n_samples=1000):

    clear_queries = [
        "What is the capital of France?",
        "How do I convert a pandas DataFrame to a CSV file?",
        "Show me the recipe for chocolate chip cookies.",
        "Explain the process of photosynthesis in detail.",
        "What was the significance of the Magna Carta?",
        "Write a Python function to calculate Fibonacci numbers.",
        "Summarize the plot of The Great Gatsby by F. Scott Fitzgerald.",
        "List five exercises for strengthening core muscles.",
        "What temperature should I cook chicken to ensure it's safe?",
        "Explain the difference between HTTP and HTTPS protocols.", ## I added more below
        "Which is healthier: running or swimming?",
        "What are the steps to apply for a US visa?",
        "What lessons can businesses learn from the 2008 financial crisis?",
        "What would happen if the Internet shut down for a week?",
        "How old was Alexander the Great when he came into power?",
        "When did Germany become a country?"
        "How do I change a tire when I have no car jack?",
        "What are the most effective study tactics for students with ADHD?",
        "What meals are good for muscle recovery?"
    ]
    
    ambiguous_queries = [
        "Why is it doing that?",
        "Can you help me with this?",
        "Is it good?",
        "What do they mean?",
        "How does it work?",
        "Can you tell me more about that topic?",
        "Why isn't it working?",
        "What's the best one?",
        "How do I fix it?",
        "When should I use it?", ## added more below
        "What are the best exercises for legs?",
        "What is the best workout meal?",
        "What is the best studying technique?",
        "What car should I buy?",
        "When will it be over?",
        "Who made the wall?",
        "Why calculators?",
        "How is Antartica?",
        "When Muhammad Ali fought him, did he win?",
        "Should I do it now or later?",
        "Is this one better?",
        "Tell me more about that."
    ]
    

    np.random.seed(42)
    clear_expanded = []
    for query in clear_queries:
        clear_expanded.append(query)
        words = query.split()
        for _ in range(9):  # Create 9 variations of each template

            if len(words) > 5:
                modified = ' '.join(words[:len(words)-np.random.randint(1, 3)]) + ' ' + ' '.join(words[-np.random.randint(1, 3):])
                clear_expanded.append(modified)
            else:
                clear_expanded.append(query)
    
    ambiguous_expanded = []
    for query in ambiguous_queries:
        ambiguous_expanded.append(query)
        
        #adding vague endings
        vague_endings = ["or something", "maybe", "I guess", "I think", "..."]
        fillers = ['um', 'like', 'so', 'you know', 'I mean']

        for _ in range(9):  # Create 9 variations of each template
            # Might add filler words but keep ambiguous
            rand_num = np.random.random()
            if rand_num > 0.6:
                ambiguous_expanded.append(np.random.choice(fillers) + " " + query)
            elif rand_num > 0.3:
                ambiguous_expanded.append(query + " " + np.random.choice(vague_endings))
            else: 
                truncated = " ".join(query.split()[:-1])
                ambiguous_expanded.append(truncated if truncated else query)
    
    all_texts = clear_expanded + ambiguous_expanded
    labels = [0] * len(clear_expanded) + [1] * len(ambiguous_expanded)
    

    indices = np.arange(len(all_texts))
    np.random.shuffle(indices)
    
    return [all_texts[i] for i in indices[:n_samples]], [labels[i] for i in indices[:n_samples]]


def main():

    texts, labels = generate_training_data(500)
    
    # Train ambiguity detector
    detector = AmbiguityDetector()
    detector.train(texts, labels)

    detector.save_model("model.pkl")
    

    test_queries = [
        "How did Mark Zuckerberg become rich?", # clear
        "How long will a flight take?", # ambiguous
        "What's the best field?", # ambiguous
        "When will this end?", # ambiguous
        "What's the most effective calves workout?", # clear
        "What is something useful I can buy with $5" # clear
    ]
    
    for query in test_queries:
        prediction, probabilities = detector.predict(query)
        ambiguity_analysis = detector.analyze_ambiguity(query)
        
        print(f"\nQuery: '{query}'")
        print(f"Prediction: {'Ambiguous' if prediction[0] == 1 else 'Clear'}")
        print(f"Confidence: {max(probabilities[0]) * 100:.2f}%")
        
        if ambiguity_analysis['ambiguity_types']:
            print("Ambiguity Analysis:")
            for i, (ambiguity_type, explanation) in enumerate(zip(
                ambiguity_analysis['ambiguity_types'], 
                ambiguity_analysis['explanations']
            )):
                print(f"  {i+1}. {ambiguity_type}: {explanation}")


if __name__ == "__main__":
    main()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        44

    accuracy                           1.00        82
   macro avg       1.00      1.00      1.00        82
weighted avg       1.00      1.00      1.00        82

Model saved to model.pkl

Query: 'How did Mark Zuckerberg become rich?'
Prediction: Clear
Confidence: 65.00%
Ambiguity Analysis:
  1. Lexical Ambiguity: Words with multiple meanings: rich
  2. Incomplete Information: Query appears too short and may lack sufficient context

Query: 'How long will a flight take?'
Prediction: Ambiguous
Confidence: 69.00%
Ambiguity Analysis:
  1. Lexical Ambiguity: Words with multiple meanings: flight
  2. Incomplete Information: Query appears too short and may lack sufficient context

Query: 'What's the best field?'
Prediction: Ambiguous
Confidence: 99.00%
Ambiguity Analysis:
  1. Lexical Ambiguity: Words with multiple meanings: best, 